In [12]:
%load_ext autoreload
%autoreload 2
from letters_dataset import LettersDataset
from accio import Accio
from torch.utils import data
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
batch_size = 64
device = torch.device("cpu")
model = Accio(input_size=41,output_size=15,device=device)
model.load_state_dict(torch.load('./models/accio_epoch_19.pth'))

<All keys matched successfully>

In [14]:
val_dataset = LettersDataset('clean_out/X_demo.csv', 'clean_out/Y_demo.csv',val_mode=True, device=device)  
val_loader = data.DataLoader(val_dataset,  batch_size=batch_size)

w = 190


In [15]:
# evaluaate accuracy on validation set
model.eval()
letter_haraka = []

line_constructed = ''

with torch.no_grad():
    for (X_batch,y_batch) in val_loader:
        # y_pred = model(X_batch)['diacritics']
        y_pred = model(X_batch)
        # we transpose because the loss function expects the second dimension to be the classes
        # y_pred is now (batch_size, n_classes, seq_len)
        y_pred = y_pred.transpose(1, 2)
        _, predicted = torch.max(y_pred.data, 1)
        # Count only non-padding characters
        for x,y in zip(X_batch,predicted):
            for xx,yy in zip(x,y):
                # we reached the end of the sentence
                # print(xx.item())
                # print(val_dataset.char_encoder.get_pad_id())
                # print(val_dataset.char_encoder.get_id_by_token(UNK_TOKEN))
                if xx.item() == val_dataset.char_encoder.get_pad_id():
                    break
                
                line_constructed += val_dataset.char_encoder.get_token_by_id(xx.item())
                ll = val_dataset.char_encoder.is_arabic_letter(xx.item())
                
                if ll:
                    letter_haraka.append([ll,yy.item()])
                    line_constructed += val_dataset.harakat_encoder.get_token_by_id(yy.item())


# for l in letter_haraka:
#     line_constructed += l[0]
#     line_constructed += val_dataset.harakat_encoder.get_token_by_id(l[1])
print(line_constructed)

صُعِدْ أَدْهَمَ وَحَمْزَةَ الشَّجَرَةِ حَتَّى يَقْطَعَا الطَّرِيقَ عَلَى الْأَشْرَارِ وَيُنْقِّذَا الْأَمِيرَةَ وَلَكِنْ لَمْ يُنَجِّحَا فِي ذَلِكَ وَلَكِنَّهُمَا تَمَكَّنَا مِنْ الْهَرَبِ وَالْعَوْدَةِ إِلَى الْقَرْيَةِ وَتَقَرَّرُ الْأَمِيرَةُ الزَّوَاجُ مِنْ أَدْهِمَ وَتَعَيُّشًا سَعِيدَيْنِ مَعًا، وَلَكِنْ قَالَ حَمْزَةُ<unk>لَا يُوجَدُ أَشْرَارٌ عَلَى الشَّجَرَةِ
